In [43]:
import pandas as pd
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split
from PIL import Image
import cv2
import glob
import os
import pickle
import xml.etree.ElementTree as ET
from os import listdir, getcwd
from os.path import join
import struct
from tqdm import tqdm

In [44]:
def get_table_bbox(xml, width, height):
    
    tree = ET.parse(xml)
    root = tree.getroot()
    
    left, bottom, right, top = list(map(lambda x: struct.unpack('!d', bytes.fromhex(x))[0], root.get("CropBox").split()))
    # print(left, bottom, right, top)
    pa_width = np.abs(left - right)
    pa_height = np.abs(top - bottom)
    bboxes = []
    for each_table_comp in tree.findall(".//Composite[@Label='TableBody']"):
        t_left, t_bottom, t_right, t_top = list(map(lambda x: struct.unpack('!d', bytes.fromhex(x))[0], each_table_comp.get("BBox").split()))
        t_left = int(width/pa_width * np.abs(t_left - left))
        t_right = int(width/pa_width * np.abs(t_right - left))
        t_bottom = int(height/pa_height * np.abs(t_bottom - bottom)) 
        t_top = int(height/pa_height * np.abs(t_top - bottom))
        bboxes.append([t_left, t_top, t_right, t_bottom])
    return bboxes

In [45]:
def create_mask(new_h, new_w, bboxes = None):
 
    mask = np.zeros((new_h, new_w), dtype=np.uint8)
    # print(bboxes)
    if bboxes is None or len(bboxes)==0:
         return Image.fromarray(mask.astype(np.uint8),'L')
    
    for box in bboxes:
        mask[box[3]:box[1], box[0]:box[2]] = 255
    return Image.fromarray(mask.astype(np.uint8),'L')

In [47]:
images_dir = os.path.join('marmot_dataset_v1.0', 'data', 'English', 'Positive', 'Raw')
img_file_list = [i for i in os.listdir(images_dir) if i.endswith('bmp')]
new_h = 512
new_w = 512
for each_image in tqdm(img_file_list):
    img_path = os.path.join(images_dir, each_image)
    image = Image.open(img_path)
    table_annot = os.path.join('marmot_dataset_v1.0', 'data', 'English', 'Positive', 'Labeled', each_image.replace('bmp', 'xml'))
    image = image.resize((new_h, new_w))
    if image.mode != 'RGB':
        image = image.convert("RGB")
    
    table_bboxes = get_table_bbox(table_annot, 512, 512)
    table_mask = create_mask(new_h, new_w, table_bboxes)
    
    save_image_path = os.path.join('Dataset', 'images', each_image.replace('bmp', 'jpeg'))
    save_table_mask_path = os.path.join('Dataset', 'table mask', each_image.replace('bmp', 'jpeg'))
    
    image.save(save_image_path)
    table_mask.save(save_table_mask_path)

100%|██████████| 509/509 [00:41<00:00, 12.36it/s]


In [50]:
images_dir = os.path.join('marmot_dataset_v1.0', 'data', 'English', 'Negative', 'Raw')
img_file_list = [i for i in os.listdir(images_dir) if i.endswith('bmp')]

for each_image in tqdm(img_file_list):
    img_path = os.path.join(images_dir, each_image)
    image = Image.open(img_path)
    image = image.resize((new_h, new_w))
    if image.mode != 'RGB':
        image = image.convert("RGB")
    table_mask = create_mask(new_h, new_w)
    save_image_path = os.path.join('Dataset', 'images', each_image.replace('bmp', 'jpeg'))
    save_table_mask_path = os.path.join('Dataset', 'table mask', each_image.replace('bmp', 'jpeg'))
    
    image.save(save_image_path)
    table_mask.save(save_table_mask_path)


100%|██████████| 484/484 [00:24<00:00, 19.53it/s]
